In [1]:
!pip install jupyter_black
import jupyter_black
jupyter_black.load()

import pandas as pd
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
thinc 8.3.2 re

## Задача 4: Split

Реализуйте разбиение датасета на train, test и val при помощи pandas и без использования циклов на Python. Разбиение должно быть стратифицировано по колонкам, данные должны быть перемешаны. Подробно объясните и/или прокомментируйте, почему ваш код делает то, что нужно.

In [2]:
df = pd.read_csv(
    "/kaggle/input/passenger-list-for-the-estonia-ferry-disaster/estonia-passenger-list.csv"
)

In [3]:
def split_stratified(df, stratify_columns, train_frac=0.6, val_frac=0.2):
    # Перемешиваем данные
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    # Доля тестовой выборки
    test_frac = 1 - train_frac - val_frac

    # Группируем данные по колонкам для стратификации.
    # Таким образом, мы будем знать соотношение между числом значений признаков.
    # Например, группировка по ['Category', 'Survived'] даст нам таблицу, из которой
    # мы узнаем соотношение числа виживших ко всем поссажирам в C и P классах отдельно,
    # что позволит нам верно стратифицировать
    grouped = df.groupby(stratify_columns)

    # Создаем индексы для каждого элемента в каждой группе
    df["group_index"] = grouped.cumcount()

    # Считаем количество элементов в каждой группе
    group_sizes = grouped["group_index"].transform("max") + 1

    # Рассчитываем индексы разбиения для train, val и test
    train_index = (train_frac * group_sizes).astype(int)
    val_index = ((train_frac + val_frac) * group_sizes).astype(int)

    # Создаем отдельный столбец, где присваиваем каждой строке метку (train, val, test)
    # на основе её индекса
    df["split"] = np.where(
        df["group_index"] < train_index,
        "train",
        np.where(df["group_index"] < val_index, "val", "test"),
    )

    # Разбиваем датафрейм на три части по меткам, используя срез с условием
    train_df = df[df["split"] == "train"].drop(columns=["group_index", "split"])
    val_df = df[df["split"] == "val"].drop(columns=["group_index", "split"])
    test_df = df[df["split"] == "test"].drop(columns=["group_index", "split"])

    return train_df, val_df, test_df

In [4]:
train, val, test = split_stratified(df, ["Category", "Survived"])

Из таблиц ниже можно заметить, что мы разбили датасет на три части так, что соотношения категориальных значений признаков соблидается. Т.е. в целом датасете в категории C соотношение выживших равно 39/154=0.25. В частях, на которые мы разбили 23/92=8/31=8/31~0.25. Аналогично для категории P. Это доказывает стратифицированность данных.

In [5]:
df.groupby(["Category", "Survived"]).count()

PassengerId  Country  Firstname  Lastname  Sex  Age
Category Survived                                                     
C        0                 154      154        154       154  154  154
         1                  39       39         39        39   39   39
P        0                 698      698        698       698  698  698
         1                  98       98         98        98   98   98

In [6]:
train.groupby(["Category", "Survived"]).count()

PassengerId  Country  Firstname  Lastname  Sex  Age
Category Survived                                                     
C        0                  92       92         92        92   92   92
         1                  23       23         23        23   23   23
P        0                 418      418        418       418  418  418
         1                  58       58         58        58   58   58

In [7]:
val.groupby(["Category", "Survived"]).count()

PassengerId  Country  Firstname  Lastname  Sex  Age
Category Survived                                                     
C        0                  31       31         31        31   31   31
         1                   8        8          8         8    8    8
P        0                 140      140        140       140  140  140
         1                  20       20         20        20   20   20

In [8]:
test.groupby(["Category", "Survived"]).count()

PassengerId  Country  Firstname  Lastname  Sex  Age
Category Survived                                                     
C        0                  31       31         31        31   31   31
         1                   8        8          8         8    8    8
P        0                 140      140        140       140  140  140
         1                  20       20         20        20   20   20